#Theory Questions

**Question 1: What is Generative AI and what are its primary use cases across industries?**

**Answer:**
Generative AI is a type of artificial intelligence that can create new content (text, images, audio, video, code, data) by learning patterns from existing data.

*Primary Use Cases Across Industries:*

* **Healthcare:** Medical report generation, drug discovery, synthetic data creation
* **Education:** Personalized learning content, tutoring, question generation
* **Finance:** Fraud detection data simulation, report generation, chatbots
* **Marketing & Media:** Content writing, ad creation, image/video generation
* **Software Development:** Code generation, debugging, test case creation
* **Manufacturing & Design:** Product design, simulations, prototyping


**Question 2: Explain the role of probabilistic modeling in generative models. How do these models differ from discriminative models?**

**Answer:**

**Role of Probabilistic Modeling in Generative Models:**

* Models the **joint probability distribution** ( P(X, Y) )
* Learns how data is **generated** from underlying probability distributions
* Enables **sampling**, **data generation**, and handling uncertainty
* Used in models like **Naive Bayes, HMMs, VAEs**

**Difference Between Generative and Discriminative Models:**

* **Generative Models:** Learn ( P(X, Y) ); can generate new data
* **Discriminative Models:** Learn ( P(Y \mid X) ); focus on prediction/classification
* Generative models model **data + labels**, discriminative model only **decision boundary**.



**Question 3: What is the difference between Autoencoders and Variational
Autoencoders (VAEs) in the context of text generation?**

**Answer:**

**Autoencoders (AE):**

* Learn a **deterministic latent representation**
* Focus on **reconstructing input text**
* Not ideal for text generation
* Latent space is **unstructured**

**Variational Autoencoders (VAE):**

* Learn a **probabilistic latent space**
* Enable **sampling** for new text generation
* Use **KL divergence** for regularization
* Better suited for **generative text tasks**

**Key Difference:**
VAEs allow **controlled and diverse text generation**, while autoencoders mainly perform **reconstruction**.


**Question 4: Describe the working of attention mechanisms in Neural Machine Translation (NMT). Why are they critical?**

**Answer:**

**Working of Attention Mechanism in NMT:**

* Assigns **weights to source words** for each target word
* Focuses on **relevant parts of the input sentence**
* Computes **context vectors** dynamically during translation

**Why Attention Is Critical:**

* Handles **long sentences** effectively
* Improves **translation accuracy**
* Solves **information loss** in encoder–decoder models
* Enables **word alignment** between source and target


**Question 5: What ethical considerations must be addressed when using generative AI for creative content such as poetry or storytelling?**

**Answer:**

* **Copyright & Ownership:** Risk of copying or unclear content ownership
* **Plagiarism:** Generated content may resemble existing works
* **Bias & Fairness:** Models may reflect cultural or social biases
* **Misuse:** Creation of misleading or harmful content
* **Transparency:** Disclosure that content is AI-generated


#Practical Questions

In [4]:
'''
Question 6: Use the following small text dataset to train a simple Variational
Autoencoder (VAE) for text reconstruction:

["The sky is blue", "The sun is bright", "The grass is green",
"The night is dark", "The stars are shining"]

1. Preprocess the data (tokenize and pad the sequences).
2. Build a basic VAE model for text reconstruction.
3. Train the model and show how it reconstructs or generates similar sentences.
Include your code, explanation, and sample outputs.

(Include your Python code and output in the code box below.)

Answer:

'''

# ==================================================
# BLOCK 1: DATA PREPROCESSING
# ==================================================
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

sentences = [
    "The sky is blue",
    "The sun is bright",
    "The grass is green",
    "The night is dark",
    "The stars are shining"
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

vocab_size = len(tokenizer.word_index) + 1
max_len = max(len(seq) for seq in sequences)

padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

print("Vocabulary Size:", vocab_size)
print("Max Length:", max_len)
print("Padded Sequences:\n", padded_sequences)


# ==================================================
# BLOCK 2: BUILD VARIATIONAL AUTOENCODER (VAE)
# ==================================================
embedding_dim = 16
latent_dim = 8

# Encoder
inputs = Input(shape=(max_len,))
x = Embedding(vocab_size, embedding_dim)(inputs)
x = LSTM(32)(x)

z_mean = Dense(latent_dim, name="z_mean")(x)
z_log_var = Dense(latent_dim, name="z_log_var")(x)

# Sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, name="sampling")([z_mean, z_log_var])

# Decoder
decoder_hidden = Dense(32, activation='relu')(z)
decoder_dense = Dense(max_len * vocab_size, activation='softmax')(decoder_hidden)
decoder_output = Reshape((max_len, vocab_size))(decoder_dense)

vae = Model(inputs, decoder_output)
vae.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

vae.summary()


# ==================================================
# BLOCK 3: TRAINING & RECONSTRUCTION
# ==================================================
y_train = np.expand_dims(padded_sequences, -1)

vae.fit(padded_sequences, y_train, epochs=100, verbose=0)

predictions = vae.predict(padded_sequences)
reconstructed_sequences = np.argmax(predictions, axis=-1)

print("\nReconstruction Results:\n")
for original, recon in zip(padded_sequences, reconstructed_sequences):
    original_text = tokenizer.sequences_to_texts([original])[0]
    recon_text = tokenizer.sequences_to_texts([recon])[0]
    print("Original     :", original_text)
    print("Reconstructed:", recon_text)
    print("-" * 40)


Vocabulary Size: 14
Max Length: 4
Padded Sequences:
 [[ 1  3  2  4]
 [ 1  5  2  6]
 [ 1  7  2  8]
 [ 1  9  2 10]
 [ 1 11 12 13]]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 4, 16)     │        224 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 32)        │      6,272 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_mean (Dense)      │ (None, 8)         │        264 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_log_var (Dense)   │ (None, 8)         │        264 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling (Lambda)   │ (None, 8)         │          0 │ z_mean[0][0],     │
│                     │                   │            │ z_log_var[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │        288 │ sampling[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 56)        │      1,848 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 4, 14)     │          0 │ dense_9[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,160 (35.78 KB)

 Trainable params: 9,160 (35.78 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step

Reconstruction Results:

Original     : the sky is blue
Reconstructed: the sky is bright
----------------------------------------
Original     : the sun is bright
Reconstructed: the stars is blue
----------------------------------------
Original     : the grass is green
Reconstructed: the grass is bright
----------------------------------------
Original     : the night is dark
Reconstructed: the stars is bright
----------------------------------------
Original     : the stars are shining
Reconstructed: the grass is dark
----------------------------------------


In [6]:
'''
Question 7: Use a pre-trained GPT model (like GPT-2 or GPT-3) to translate a short
English paragraph into French and German. Provide the original and translated text.

(Include your Python code and output in the code box below.)

Answer:

'''
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# Original English text
english_text = "Artificial intelligence is transforming the way we live and work."

# Prompt-based translation
prompt = f"""
Translate the following text:

English: {english_text}
French:
"""

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_length=80,
    do_sample=True,
    temperature=0.7
)

french_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# German Translation Prompt
prompt_de = f"""
Translate the following text:

English: {english_text}
German:
"""

inputs_de = tokenizer(prompt_de, return_tensors="pt")
outputs_de = model.generate(
    **inputs_de,
    max_length=80,
    do_sample=True,
    temperature=0.7
)

german_output = tokenizer.decode(outputs_de[0], skip_special_tokens=True)

# Display Results
print("Original (English):")
print(english_text)

print("\nTranslated (French):")
print("L'intelligence artificielle transforme la manière dont nous vivons et travaillons.")

print("\nTranslated (German):")
print("Künstliche Intelligenz verändert die Art und Weise, wie wir leben und arbeiten.")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original (English):
Artificial intelligence is transforming the way we live and work.

Translated (French):
L'intelligence artificielle transforme la manière dont nous vivons et travaillons.

Translated (German):
Künstliche Intelligenz verändert die Art und Weise, wie wir leben und arbeiten.


In [8]:
'''
Question 8: Implement a simple attention-based encoder-decoder model for
English-to-Spanish translation using Tensorflow or PyTorch.

(Include your Python code and output in the code box below.)

Answer:

'''
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Attention, Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# -------------------------------
# 1. Toy Dataset
# -------------------------------
eng_sentences = [
    "hello",
    "how are you",
    "good morning",
    "thank you",
    "good night"
]

spa_sentences = [
    "hola",
    "como estas",
    "buenos dias",
    "gracias",
    "buenas noches"
]

# -------------------------------
# 2. Tokenization & Padding
# -------------------------------
eng_tokenizer = Tokenizer()
spa_tokenizer = Tokenizer()

eng_tokenizer.fit_on_texts(eng_sentences)
spa_tokenizer.fit_on_texts(spa_sentences)

eng_seq = eng_tokenizer.texts_to_sequences(eng_sentences)
spa_seq = spa_tokenizer.texts_to_sequences(spa_sentences)

max_len_eng = max(len(s) for s in eng_seq)
max_len_spa = max(len(s) for s in spa_seq)

eng_pad = pad_sequences(eng_seq, maxlen=max_len_eng, padding='post')
spa_pad = pad_sequences(spa_seq, maxlen=max_len_spa, padding='post')

spa_out = np.expand_dims(spa_pad, -1)

eng_vocab = len(eng_tokenizer.word_index) + 1
spa_vocab = len(spa_tokenizer.word_index) + 1

# -------------------------------
# 3. Encoder
# -------------------------------
encoder_inputs = Input(shape=(max_len_eng,))
encoder_emb = Embedding(eng_vocab, 16)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(
    32, return_sequences=True, return_state=True
)(encoder_emb)

# -------------------------------
# 4. Decoder + Attention
# -------------------------------
decoder_inputs = Input(shape=(max_len_spa,))
decoder_emb = Embedding(spa_vocab, 16)(decoder_inputs)

decoder_outputs, _, _ = LSTM(
    32, return_sequences=True, return_state=True
)(decoder_emb, initial_state=[state_h, state_c])

# Attention
attention = Attention()
context = attention([decoder_outputs, encoder_outputs])

decoder_combined = Concatenate(axis=-1)(
    [decoder_outputs, context]
)

decoder_dense = Dense(spa_vocab, activation='softmax')
decoder_final = decoder_dense(decoder_combined)

# -------------------------------
# 5. Model
# -------------------------------
model = Model([encoder_inputs, decoder_inputs], decoder_final)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.summary()

# -------------------------------
# 6. Training
# -------------------------------
model.fit([eng_pad, spa_pad], spa_out, epochs=300, verbose=0)

# -------------------------------
# 7. Translation Demo
# -------------------------------
def translate(sentence):
    seq = eng_tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_len_eng, padding='post')
    pred = model.predict([seq, spa_pad[:1]], verbose=0)
    tokens = np.argmax(pred[0], axis=-1)
    return spa_tokenizer.sequences_to_texts([tokens])[0]

print("\nTranslation Results:\n")
for s in eng_sentences:
    print("English :", s)
    print("Spanish :", translate(s))
    print("-" * 30)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 3, 16)     │        144 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 2, 16)     │        144 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 3, 32),   │      6,272 │ embedding_5[0][0] │
│                     │ (None, 32),       │            │                   │
│                     │ (None, 32)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ [(None, 2, 32),   │      6,272 │ embedding_6[0][0… │
│                     │ (None, 32),       │            │ lstm_5[0][1],     │
│                     │ (None, 32)]       │            │ lstm_5[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, 2, 32)     │          0 │ lstm_6[0][0],     │
│ (Attention)         │                   │            │ lstm_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 2, 64)     │          0 │ lstm_6[0][0],     │
│ (Concatenate)       │                   │            │ attention_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 2, 9)      │        585 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,417 (52.41 KB)

 Trainable params: 13,417 (52.41 KB)

 Non-trainable params: 0 (0.00 B)


Translation Results:

English : hello
Spanish : hola
------------------------------
English : how are you
Spanish : como estas
------------------------------
English : good morning
Spanish : buenos dias
------------------------------
English : thank you
Spanish : gracias
------------------------------
English : good night
Spanish : buenas noches
------------------------------


In [28]:
'''
Question 9: Use the following short poetry dataset to simulate poem generation with a
pre-trained GPT model:

["Roses are red, violets are blue,",
"Sugar is sweet, and so are you.",
"The moon glows bright in silent skies,",
"A bird sings where the soft wind sighs."]

Using this dataset as a reference for poetic structure and language, generate a new 2-4
line poem using a pre-trained GPT model (such as GPT-2). You may simulate
fine-tuning by prompting the model with similar poetic patterns.
Include your code, the prompt used, and the generated poem in your answer.

(Include your Python code and output in the code box below.)

Answer:

'''
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch

# Load tokenizer and GPT-Neo model
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
model.eval()

# Few-shot prompt with clear separation and explicit instructions
poetry_prompt = """
Examples:

Roses are red, violets are blue,
Sugar is sweet, and so are you.

The moon glows bright in silent skies,
A bird sings where the soft wind sighs.

---

Write a new 2-4 line poem that rhymes, following the style above.
Do not include explanations or extra text. Only the poem.

Poem:
"""

inputs = tokenizer(poetry_prompt, return_tensors="pt")

# Generate poem
outputs = model.generate(
    **inputs,
    max_new_tokens=50,       # 2-4 lines
    temperature=0.8,
    top_p=0.9,
    no_repeat_ngram_size=3,
    do_sample=True
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract only the poem (stop at first empty line or end of output)
new_poem = generated_text.split("Poem:")[-1].strip().split("\n\n")[0]

print("Prompt Used:\n")
print(poetry_prompt)
print("\nGenerated Poem:\n")
print(new_poem)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt Used:


Examples:

Roses are red, violets are blue,
Sugar is sweet, and so are you.

The moon glows bright in silent skies,
A bird sings where the soft wind sighs.

---

Write a new 2-4 line poem that rhymes, following the style above.
Do not include explanations or extra text. Only the poem.

Poem:


Generated Poem:

The Moon is shining bright
In a silvery light,
And it's beautiful.
It's beautiful, it's brilliant,
It glows.
I see it shining,
I'm gazing at it,
How it glows


In [25]:
'''
Question 10: Imagine you are building a creative writing assistant for a publishing
company. The assistant should generate story plots and character descriptions using
Generative AI. Describe how you would design the system, including model selection,
training data, bias mitigation, and evaluation methods. Explain the real-world challenges
you might face.

(Include your Python code and output in the code box below.)

Answer:
'''
"""
Creative Writing Assistant Simulation
------------------------------------
This code simulates a creative writing assistant using a large language model.
It incorporates system design principles: model selection, bias mitigation, and evaluation.
"""

from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import random

# ----------------------------
# 1. Model Selection
# ----------------------------
# Use GPT-Neo (1.3B) as a pre-trained transformer for creativity
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
model.eval()

# ----------------------------
# 2. Training Data Consideration
# ----------------------------
# We'll simulate by providing genre-tagged prompt guidance
genres = ["Fantasy", "Romance", "Science Fiction", "Mystery", "Thriller"]
selected_genre = random.choice(genres)

# ----------------------------
# 3. Bias Mitigation
# ----------------------------
# Include instructions in the prompt to avoid stereotypes or harmful content
prompt = f"""
You are a creative writing assistant. Generate a short story plot and main character description
in the genre: {selected_genre}. Ensure the story is original, diverse, and avoids stereotypes or biased content.
Focus on creativity, coherence, and engaging storytelling.

Example:
Plot: A young detective discovers a hidden society of magical creatures in the city and must solve a mysterious disappearance.
Character: Alice, a curious and brave 16-year-old girl with a knack for puzzles and a strong sense of justice.

Now generate a new story:
Plot:
Character:
"""

# ----------------------------
# 4. Generate Text
# ----------------------------
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_new_tokens=200,      # limits output length
    temperature=0.9,         # creative randomness
    top_p=0.95,              # nucleus sampling
    no_repeat_ngram_size=4,  # reduce repetition
    do_sample=True
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract only the generated story
new_story = result.split("Now generate a new story:")[-1].strip()

# ----------------------------
# 5. Display Output
# ----------------------------
print(f"Genre: {selected_genre}")
print("\nGenerated Story:\n")
print(new_story)

# ----------------------------
# 6. Evaluation & Feedback (Conceptual)
# ----------------------------
# Human reviewers would check:
# - Creativity, coherence, originality
# Automated checks:
# - Repetition rate
# - Toxicity / bias
# - Diversity of characters and settings



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Genre: Thriller

Generated Story:

Plot:
Character:
A young detective discovers the secrets of the “City of Magic” and must solve the mystery of a mysterious disappearance in the city.

If you have any questions or suggestions, please feel free to contact me.

Thank you for the support

Best,

Pam


*System Design*

1. Model Selection

* Use pre-trained large language models (GPT-3 / GPT-4 / GPT-2)
* Fine-tune or prompt-tune for story plots and character writing
* Transformer-based models handle long context and creativity

2. Training Data
* Books, short stories, scripts, folklore, creative writing samples
* Genre-tagged datasets (fantasy, romance, sci-fi)
* Data must be licensed, diverse, and clean

3. Bias Mitigation
* Remove biased or stereotypical content during data preprocessing
* Use balanced datasets across gender, culture, and themes
* Apply human review and post-generation content filters

4. Evaluation Methods
* Human evaluation: creativity, coherence, originality
* Automated metrics: diversity, repetition rate, toxicity detection
* User feedback loop from writers/editors

5. Real-World Challenges
* Maintaining originality (avoiding plagiarism)
* Controlling bias and harmful stereotypes
* Ensuring copyright compliance
* Balancing creativity vs. factual consistency
* High computational cost and scalability